# Exercice Prénoms

Le but de cet exercice est d'utiliser les graphiques que nous avons précédemment vu sur notre dataset de prénoms ensemble, je coderai, et vous vous poserez les bonnes questions, dans une bonne démarche de data-scientifiques.

Quelle est l'évolution de tel prénom ? Quels sont les prénoms les plus populaires de l'année 2016 ?... Il y a plein de questions que nous pouvons nous poser et associer à un graphique.

On va reprendre notre dataset sur les naissances en France. Vous pouvez le récupérer ici :
- [Télécharger dataset des prénoms](https://github.com/DanYellow/cours/blob/main/big-data-s4/datasets/naissances-par-departement-1900-2020.zip) (Cliquez sur le bouton "Télécharger / Download" sur github)
  - Décompressez l'archive 

# Pour les utilisateurs de Google colab

Petit apparté pour les utilisateurs de google colab. Pour charger un fichier local, il faudra rajouter les lignes de codes suivantes :

```python
from google.colab import files
uploaded = files.upload()

import io
import pandas as pd
# Très important : le nom du fichier passé en paramètre de la fonction "uploaded" doit avoir le même nom que le fichier que vous avez uploadé sinon, 
# vous aurez forcément une erreur
df = pd.read_csv(io.BytesIO(uploaded['nom-du-fichier-uploader.csv']))
```
- **Ces lignes doivent être avant la manipulation d'un DataFrame et de préférence dans une cellule dédiée pour éviter d'uploader votre fichier à chaque fois**
- **Vous ne pouvez pas importer de fichiers en navigation privée**

- [Voir plus d'informations sur le chargement de fichiers externes avec Google colab - anglais](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92)

In [1]:
# On importe les bibliothèques dont nous aurons besoin
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random

# liste_prenoms_source = pd.read_csv("A-REMPLACER", sep=";")
liste_prenoms_source = pd.read_csv("_tmp-dpt2019.csv", sep=";")

# Exploration des données

In [12]:
# Aperçu du DataFrame
display(liste_prenoms_source.head(3))

# nbre lignes / colonnes
display(liste_prenoms_source.shape)

# Type des colonnes
display(liste_prenoms_source.dtypes)

,sexe,preusuel,annais,dpt,nombre
0,1,_PRENOMS_RARES,1900,02,7
1,1,_PRENOMS_RARES,1900,04,9
2,1,_PRENOMS_RARES,1900,05,8


(3676682, 5)

sexe         int64
preusuel    object
annais      object
dpt         object
nombre       int64
dtype: object

# Nettoyage des données
Déjà fait pour vous. Il est dans la cellule ci-dessous.

In [41]:
liste_prenoms_nettoyage = liste_prenoms_source.copy()

# On supprime les colonnes avec les lignes ayant des données manquantes
liste_prenoms_nettoyage.dropna(inplace = True) 

# On supprime les lignes avec la valeur "_PRENOMS_RARES" pour la colonne "preusuel" ou
# XXXX comme valeur pour l'année ou 
# XX comme valeur pour le département
liste_prenoms_nettoyage = liste_prenoms_source[
       (liste_prenoms_source["preusuel"] != "_PRENOMS_RARES") &
        (liste_prenoms_source["annais"] != "XXXX") &
    (liste_prenoms_source["dpt"] != "XX")
]
# Il est tout à fait possible de nettoyer d'une autre façon, nous aurions pu, par exemple, remplacer 
# les années XXXX par une année définie si ceci ne vous dérange pas. 

# La colonne "sexe" est un entier, dans notre cas ceci n'a pas de sens. Transformer en entier la colonne.
liste_prenoms_nettoyage['sexe'] = liste_prenoms_nettoyage['sexe'].astype(str)

liste_prenoms_df = liste_prenoms_nettoyage.copy().reset_index(drop=True)

display(liste_prenoms_df)

,sexe,preusuel,annais,dpt,nombre
0,1,AADIL,1983,84,3
1,1,AADIL,1992,92,3
2,1,AAHIL,2016,95,3
3,1,AARON,1962,75,3
4,1,AARON,1976,75,3
...,...,...,...,...,...
3618396,2,ZYA,2013,44,4
3618397,2,ZYA,2013,59,3
3618398,2,ZYA,2017,974,3
3618399,2,ZYA,2018,59,3


# Ajout de données
Pour voir plus de données et donc gérer plus de graphiques, nous allons ajouter des une colonne "traits_personnalite" à notre DataFrame.

In [44]:
# Rajout d'une colonne concernant les traits de personnalité liés à une personne.
# Ces traits de personnalité seront ajoutés de façon aléatoires, nous n'aurons donc tous pas
# les mêmes résultats
liste_traits_personnalite = ["Bienveillance", "Empathique", 
                             "Téméraire", "Gentillesse", 
                             "Romantique", "Têtu(e)", "Déterminé(e)",
                             "Drôle", "Naïf/Naïve", "Charismatique",
                             "Aventureux/Aventureuse", "Autonome", "Courtois(e)",
                             "Créatif/Créative", "Efficace", "Amical(e)",
                             "Intuitif/Intuitive", "Méticuleux/Méticuleuse", "Persuasif/Persuasive",
                             "Bavard(e)", "Patient(e)", "Travailleur/Travailleuse",
                             "Paresseux/Paresseuse", "Sérieux/Sérieuse", "Patient(e)", 
                             "Vif/Vive", "Perceptif/Perceptive", "Pudique",
                             "Impulsive/Impulsif", "Timide", "Respecteux/Respecteuse",
                             "Bienveillant(e)", "Extraverti(e)", "Consciencieux/Conscieuseuse"
                            ]

colonne_traits_personnalite = np.random.choice(liste_traits_personnalite, size=(len(liste_prenoms_df)))
colonne_traits_personnalite = [
    f"{valeur}, {colonne_traits_personnalite[-(idx + 1)]}" for idx, valeur in enumerate(colonne_traits_personnalite)
]

liste_prenoms_df['traits_personnalite'] = colonne_traits_personnalite

display(liste_prenoms_df) 

,sexe,preusuel,annais,dpt,nombre,traits_personnalite
0,1,AADIL,1983,84,3,"Bienveillant(e), Créatif/Créative"
1,1,AADIL,1992,92,3,"Gentillesse, Courtois(e)"
2,1,AAHIL,2016,95,3,"Empathique, Téméraire"
3,1,AARON,1962,75,3,"Bienveillant(e), Bavard(e)"
4,1,AARON,1976,75,3,"Bienveillance, Sérieux/Sérieuse"
...,...,...,...,...,...,...
3618396,2,ZYA,2013,44,4,"Sérieux/Sérieuse, Bienveillance"
3618397,2,ZYA,2013,59,3,"Bavard(e), Bienveillant(e)"
3618398,2,ZYA,2017,974,3,"Téméraire, Empathique"
3618399,2,ZYA,2018,59,3,"Courtois(e), Gentillesse"


# Exercice 1 - Courbe (line chart)

Afficher un graphique de type ligne. Posons-nous des questions !

In [2]:
# Notre code

# On n'oublie pas de définir la taille de notre graphique. A adapter en fonction de nos besoins
plt.figure(figsize=(15, 6), dpi=144)

# Exercice 2 - Diagramme en bâtons

Afficher un graphique de type bâtons. Posons-nous des questions !

In [ ]:
# Notre code

# Exercice 3 - Diagramme circulaire

Afficher un graphique de type bâtons. Posons-nous des questions !

In [50]:
# Notre code